# BlazingSQL vs. Apache Spark 

Below we have one of our popular workloads running with [BlazingSQL + RAPIDS AI](https://blazingdb.com) and then running the entire ETL phase again, only this time with Apache Spark + PySpark.

In this notebook, we will cover: 
- How to set up [BlazingSQL](https://blazingsql.com) and the [RAPIDS AI](https://rapids.ai/) suite.
- How to read and query csv files with cuDF and BlazingSQL.
- How BlazingSQL compares against Apache Spark (analyzing over 20M records).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-39814657-5&cid=555&t=event&ec=guides&ea=bsql_vs_spark&dt=bsql_vs_spark)

## Setup
### Environment Sanity Check 

RAPIDS packages (BlazingSQL included) require Pascal+ architecture to run. For Colab, this translates to a T4 GPU instance. 

The cell below will let you know what type of GPU you've been allocated, and how to proceed.

In [4]:
# tag specs
colab_smi = !nvidia-smi

# focus GPU type
try:
    my_gpu = ' '.join(colab_smi[7].split()[2:4])
# not on gpu acceleration 
except:
    raise Exception("\nPlease make sure you've configured Colab to request a GPU instance type.\n\n"
                    "At top of Colab, try: Runtime -> Change runtime type -> Hardware accelerator -> GPU -> Save\n")

# not allocated compatable GPU
if (my_gpu != b'Tesla T4') and (my_gpu != 'Tesla P100-PCIE...') and (my_gpu != 'GeForce GTX'):
    # allocated K80
    if my_gpu == 'Tesla K80':
        raise Exception("\nYou've been allocated a K80 instance\n\n"
                    "Unfortunately, this demo requires a T4 instance\n\n"
                    "At top of Colab, try: Runtime -> Reset all runtimes...\n")
    else:
        raise Exception(f"\nYou've achieved wizardy.\nyour GPU is {my_gpu}\nPlease inform info@blazingsql.com")

# allocated compatable GPU
else:
    print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


## Installs 

Below you will find three code blocks:
1. The first installs miniconda.
2. The second installs RAPIDS AI and sets up the system environment. 
3. The third installs BlazingSQL.

### Miniconda

In [0]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

### RAPIDS AI

In [ ]:
# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c nvidia -c rapidsai \
  -c numba -c conda-forge -c pytorch -c defaults \
  cudf=0.9 cuml=0.9 cugraph=0.9 python=3.6 cudatoolkit=10.0

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
    shutil.copy('/usr/local/lib/'+fn, os.getcwd())

### BlazingSQL

In [ ]:
# Install BlazingSQL for CUDA 10.0
! conda install -q -y --prefix /usr/local -c conda-forge -c defaults -c nvidia -c rapidsai \
   -c blazingsql/label/cuda10.0 -c blazingsql \
   blazingsql-calcite blazingsql-orchestrator blazingsql-ral blazingsql-python

!pip install flatbuffers

## Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context (i.e. where information such as FileSystems you have registered and Tables you have created will be stored). If you have issues running this cell, restart runtime and try running it again.

In [5]:
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

BlazingContext ready


### Load & Query Table

In [11]:
!wget https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv

In [7]:
%%time
#Load CSVs into GPU DataFrames (gdf)
netflow_gdf = cudf.read_csv('nf-chunk2.csv')

CPU times: user 156 ms, sys: 132 ms, total: 288 ms
Wall time: 290 ms


In [8]:
%%time
#Create BlazingSQL Tables - There is no copy in this process
bc.create_table('netflow', netflow_gdf)

Communication error: connection to ('localhost', 8889) was lost
CPU times: user 26.6 ms, sys: 3.45 ms, total: 30 ms
Wall time: 28.8 ms


In [9]:
%%time
sql = '''
SELECT
  a.firstSeenSrcIp as source,
  a.firstSeenDestIp as destination,
  count(a.firstSeenDestPort) as targetPorts,
  SUM(a.firstSeenSrcTotalBytes) as bytesOut,
  SUM(a.firstSeenDestTotalBytes) as bytesIn,
  SUM(a.durationSeconds) as durationSeconds,
  MIN(parsedDate) as firstFlowDate,
  MAX(parsedDate) as lastFlowDate,
  COUNT(*) as attemptCount
  FROM
  main.netflow a
  GROUP BY
  a.firstSeenSrcIp,
  a.firstSeenDestIp
  '''

result = bc.sql(sql,['netflow']).get()

# print(result.columns)

result_gdf = result.columns
edges_df = result_gdf.to_pandas()
print(edges_df.head(10))

NOTE: You no longer need to send a table list to the .sql() funtion
Communication error: connection to ('localhost', 8889) was lost


TypeError: 'NoneType' object is not iterable

## Apache Spark

In [0]:
%%time
# Install Spark 
# Note: This installs Spark (version 2.4.1, as tested in April 2019)

!pip install pyspark

     |████████████████████████████████| 215.6MB 125kB/s 
     |████████████████████████████████| 204kB 44.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215964963 sha256=309c5916fd287e31dd2c8d7fd305111b567fc1f6893d5d2a51e153d98c161fba
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark
CPU times: user 1.07 s, sys: 2.32 s, total: 3.39 s
Wall time: 36.3 s


In [0]:
%%time
#I copied this cell's snippet from another Google Colab by Luca Canali here: https://colab.research.google.com/github/LucaCanali/sparkMeasure/blob/master/examples/SparkMeasure_Jupyter_Colab_Example.ipynb

from pyspark.sql import SparkSession

# Create Spark Session
# This example uses a local cluster, you can modify master to use  YARN or K8S if available 
# This example downloads sparkMeasure 0.13 for scala 2_11 from maven central

spark = SparkSession \
 .builder \
 .master("local[*]") \
 .appName("PySpark Netflow Benchmark code") \
 .config("spark.jars.packages","ch.cern.sparkmeasure:spark-measure_2.11:0.13")  \
 .getOrCreate()

CPU times: user 54.6 ms, sys: 31.3 ms, total: 86 ms
Wall time: 9.41 s


###Load & Query Table

In [0]:
%%time

netflow_df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('nf-chunk2.csv')

CPU times: user 121 ms, sys: 81.4 ms, total: 202 ms
Wall time: 5min 34s


In [0]:
%%time
netflow_df.createOrReplaceTempView('netflow')

CPU times: user 822 µs, sys: 1.43 ms, total: 2.25 ms
Wall time: 200 ms


In [0]:
%%time
sql = '''
SELECT
  a.firstSeenSrcIp as source,
  a.firstSeenDestIp as destination,
  count(a.firstSeenDestPort) as targetPorts,
  SUM(a.firstSeenSrcTotalBytes) as bytesOut,
  SUM(a.firstSeenDestTotalBytes) as bytesIn,
  SUM(a.durationSeconds) as durationSeconds,
  MIN(parsedDate) as firstFlowDate,
  MAX(parsedDate) as lastFlowDate,
  COUNT(*) as attemptCount
  FROM
  netflow a
  GROUP BY
  a.firstSeenSrcIp,
  a.firstSeenDestIp
  '''

edges_df = spark.sql(sql)

edges_df.show()

+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
|      source|    destination|targetPorts|bytesOut|bytesIn|durationSeconds|      firstFlowDate|       lastFlowDate|attemptCount|
+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
| 172.10.1.13|239.255.255.250|         15|    2975|      0|              6|2013-04-03 06:36:19|2013-04-03 06:36:27|          15|
|172.30.1.204|239.255.255.250|          8|    1750|      0|              6|2013-04-03 06:36:13|2013-04-03 06:36:20|           8|
| 172.30.2.86|      172.0.0.1|          1|     540|      0|              2|2013-04-03 06:36:09|2013-04-03 06:36:09|           1|
|172.30.1.246|      172.0.0.1|         29|    2610|   2610|              0|2013-04-03 00:26:46|2013-04-03 23:06:00|          29|
| 172.30.1.51|239.255.255.250|         16|    3850|      0|             18|2013-04-03 06:35:22|20